In [1]:
import nltk
import os
import sys
import urllib.request
import json
from pprint import pprint

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from nltk.corpus import stopwords
from collections import Counter
from bs4 import BeautifulSoup
import time

In [ ]:
#nltk.download()

In [2]:
def trans(textData): # papago 번역해주는 함수
    client_id = "VfDeHa1pQMhovCjJHYEP" # 개발자센터에서 발급받은 Client ID 값
    client_secret = "bbZZXhFBy3" # 개발자센터에서 발급받은 Client Secret 값
    encText = urllib.parse.quote(textData)
    data = "source=en&target=ko&text=" + encText
    url = "https://openapi.naver.com/v1/papago/n2mt"
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request, data=data.encode("utf-8"))
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
        res = json.loads(response_body.decode('utf-8'))
        transform = res['message']['result']['translatedText']
    else:
        print("Error Code:" + rescode)
    return transform

In [7]:
#핵심단어를 빈칸으로 바꿔서 문장 반환
def mkBlank(imWord,words):
    blankWords = []
    blankStr = ""
    blank = ""
    for word in words:
        if word == imWord:
            for i in range(len(word)):
                blank = blank + " "
            word = "(" + blank + ")"
        blankWords.append(word)
    for word in blankWords:
       blankStr = blankStr + word + " "
    return blankStr

In [78]:
mkBlank("friendship",splitText[0].split())

'The evening also featured a video about the cross-party (          ) between Biden and the late Sen '

In [9]:
originText = """The evening also featured a video about the cross-party friendship between Biden and the late Sen. John McCain, narrated by the Arizona Republican's widow, Cindy. The Senate veterans once traded barbs on foreign policy but bonded across party lines in a friendship dating to when McCain served as a military aide after returning from years as a prisoner in the Vietnam War.The former vice president delivered a powerful eulogy for his friend at McCain's memorial service after he died of brain cancer in 2018."My name is Joe Biden. I'm a Democrat. And I love John McCain," he said.Another prominent Republican -- former Secretary of State Colin Powell, who served under President George W. Bush -- offered a video testimonial about Biden as part of the Democrats' effort to draw in Republicans and independents who are frustrated with Trump's leadership and his divisive tactics."""

In [88]:
originText = """Trump's insistence that the election is somehow being rigged against him has increased in urgency as it's become more and more obvious that he is an underdog to win a second term.
That's, obviously, no coincidence. Trump has spent a lifetime projecting an air of invincibility, a narrative of winning, winning, winning. When faced with polling that suggests he isn't winning, he makes up excuses as to why.  Excuses that always have to do with some unnamed "them" who are out to get him.
While it's not ideal behavior for an adult to exhibit, when we are talking about ratings for Trump's reality show or some business bet he was on the wrong side of, the consequences aren't that terrible.
But when we are talking about the health and strength of our democracy, then what Trump is doing is catastrophically dangerous for all of us.
The Point: If Trump refuses to concede or admit defeat -- if such a defeat is clear -- it will undermine the peaceful transition of power that America has long prided itself on. And create a next four years even more polarized and awful than the last four years."""

In [89]:
splitText = originText.split('.') # 통짜리 본몬을 문장으로 나누기

In [90]:
splitText[0] # 첫번째 문장 출력

"Trump's insistence that the election is somehow being rigged against him has increased in urgency as it's become more and more obvious that he is an underdog to win a second term"

In [91]:
words = splitText[0].split() # 첫번째 문장을 띄어쓰기단위로 단어 나누기
words

["Trump's",
 'insistence',
 'that',
 'the',
 'election',
 'is',
 'somehow',
 'being',
 'rigged',
 'against',
 'him',
 'has',
 'increased',
 'in',
 'urgency',
 'as',
 "it's",
 'become',
 'more',
 'and',
 'more',
 'obvious',
 'that',
 'he',
 'is',
 'an',
 'underdog',
 'to',
 'win',
 'a',
 'second',
 'term']

In [92]:
tokens = nltk.word_tokenize(originText)

In [93]:
#nltk.pos_tag(tokens)

In [94]:
stopWords = stopwords.words('english') # 불용어 제거
stopWords.append(',')
stopWords.append('.')
stopWords.append("'s")
stopWords.append('``')
stopWords.append('--')
stopWords.append("'")
stopWords.append("''")

In [95]:
result = []
for token in tokens:
    if token.lower() not in stopWords:
        result.append(token)

In [96]:
#wordListNN = []
tags = nltk.pos_tag(result) # 명사만 골라서 넣기 ( 고유명사 제외 )
for word, tag in tags:
    if tag in ['NN','NNS']:
        wordListNN.append(word.lower())
countNN = Counter(wordListNN)

In [97]:
wordListVB = []
tags = nltk.pos_tag(result) # 동사만 골라서 넣기 
for word, tag in tags:
    if tag in ['VB','VBP','VBM','VBZ','VBD','VBN']:
        wordListVB.append(word.lower())
countVB = Counter(wordListVB)

In [104]:
countNN.most_common(100)

[('years', 3),
 ('defeat', 2),
 ('leadership', 1),
 ('tactics', 1),
 ('party', 1),
 ('veterans', 1),
 ('vice', 1),
 ('name', 1),
 ('policy', 1),
 ('love', 1),
 ('cancer', 1),
 ('brain', 1),
 ('friend', 1),
 ('widow', 1),
 ('service', 1),
 ('friendship', 1),
 ('draw', 1),
 ('eulogy', 1),
 ('effort', 1),
 ('president', 1),
 ('part', 1),
 ('independents', 1),
 ('lines', 1),
 ('insistence', 1),
 ('election', 1),
 ('urgency', 1),
 ('term', 1),
 ('spent', 1),
 ('lifetime', 1),
 ('air', 1),
 ('invincibility', 1),
 ('polling', 1),
 ('excuses', 1),
 ('adult', 1),
 ('exhibit', 1),
 ('ratings', 1),
 ('reality', 1),
 ('show', 1),
 ('business', 1),
 ('bet', 1),
 ('side', 1),
 ('consequences', 1),
 ('health', 1),
 ('strength', 1),
 ('democracy', 1),
 ('point', 1),
 ('trump', 1),
 ('admit', 1),
 ('transition', 1),
 ('power', 1)]

In [99]:
wordListNN = set(wordListNN) # set으로 바꿔 중복제거
wordListNN = list(wordListNN) # 다시 list로 변환
wordListNN

['leadership',
 'air',
 'transition',
 'ratings',
 'tactics',
 'trump',
 'exhibit',
 'party',
 'show',
 'side',
 'consequences',
 'veterans',
 'democracy',
 'vice',
 'invincibility',
 'power',
 'excuses',
 'name',
 'policy',
 'love',
 'term',
 'years',
 'adult',
 'cancer',
 'bet',
 'reality',
 'brain',
 'friend',
 'widow',
 'service',
 'lifetime',
 'polling',
 'friendship',
 'spent',
 'draw',
 'eulogy',
 'effort',
 'president',
 'part',
 'election',
 'insistence',
 'independents',
 'lines',
 'business',
 'strength',
 'defeat',
 'admit',
 'point',
 'health',
 'urgency']

In [108]:
#문장과 비교해서 빈칸뚫기

def output(impWord):
    for sentens in splitText:
        splitList = sentens.split()
        for i in splitList:
            if i == impWord:
                print(mkBlank(i,splitList))
                print(trans(sentens))

In [114]:
output(wordListNN[0])

In [113]:
for i in wordListNN:
    output(i)
    time.sleep(3)
    print()


Trump has spent a lifetime projecting an (   ) of invincibility, a narrative of winning, winning, winning 


HTTPError: HTTP Error 429: Too Many Requests